# <center>LSTM

In [1]:
import os
# os.chdir("..")

# %load_ext autoreload
# %autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.chdir("/content/drive/MyDrive/Colab_Notebooks/meson_insider")

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  after removing the cwd from sys.path.


In [5]:
data_path = Path("data")

In [6]:
example_predictions_name = "example_predictions.pkl"
test_features_name = "test_features.pkl"
train_features_name = "train_features.pkl"
train_outcomes_name = "train_outcomes.pkl"

In [7]:
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_rows', 10)

In [8]:
example_predictions = pd.read_pickle(data_path / example_predictions_name)
example_predictions.head()

g__12m_binary
date       tradingitemid               
2006-12-29 2585893             0.541588
           2585895             0.522537
           2585941             0.500847
           2585945             0.496815
           2585957             0.552594

In [9]:
test_features = pd.read_pickle(data_path /test_features_name)
train_features = pd.read_pickle(data_path / train_features_name)
train_outcomes = pd.read_pickle(data_path / train_outcomes_name)

In [10]:
from dataprocessing.process import StandardScaler, SmartLogtransformer, TrainTestSplit, Pipeline
from dataprocessing.impute import SimpleImputer

In [11]:
train_test_split = TrainTestSplit(test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(train_features, train_outcomes)

### Pipeline for data processing

In [12]:
pipe = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")), 
        ("logtransformer", SmartLogtransformer()), 
        ("scaler", StandardScaler())
    ]
)
pipe

[(imputer, <dataprocessing.impute.SimpleImp>), 
(logtransformer, <dataprocessing.process.SmartLogtransfo>), 
(scaler, <dataprocessing.process.StandardSc>)]

In [13]:
%%time
X_train = pipe.fit_transform(X_train)
X_val = pipe.transform(X_val)
X_test = pipe.transform(test_features)

CPU times: user 5.14 s, sys: 260 ms, total: 5.4 s
Wall time: 4.25 s


In [14]:
X_train

f__buytransactions_1m  f__selltransactions_1m  \
date       tradingitemid                                                  
2004-08-06 2585893                     1.124676               -0.745641   
           2585895                     1.173487                1.348587   
           2585941                    -0.821051               -0.745641   
           2585957                    -0.821051                1.268505   
           2585970                    -0.821051               -0.745641   
...                                         ...                     ...   
2012-11-23 586093745                  -0.821051               -0.745641   
           607745150                  -0.821051               -0.745641   
           610592433                  -0.821051               -0.745641   
           618650356                  -0.821051               -0.745641   
           636367352                  -0.821051               -0.745641   

                          f__buyshares_1m  f__sellshares_1m  \
date       tradingitemid                                      
2004-08-06 2585893              -0.443263          0.003256   
           2585895              -0.288378          0.003541   
           2585941               0.027006          0.003256   
           2585957               0.027006          0.003540   
           2585970               0.027006          0.003256   
...                                   ...               ...   
2012-11-23 586093745             0.027006          0.003256   
           607745150             0.027006          0.003256   
           610592433             0.027006          0.003256   
           618650356             0.027006          0.003256   
           636367352             0.027006          0.003256   

                          f__buysharespctsharesout_1m  \
date       tradingitemid                                
2004-08-06 2585893                          -0.372335   
           2585895                           0.005511   
           2585941                           0.087960   
           2585957                           0.087960   
           2585970                           0.087960   
...                                               ...   
2012-11-23 586093745                         0.087960   
           607745150                         0.087960   
           610592433                         0.087960   
           618650356                         0.087960   
           636367352                         0.087960   

                          f__sellsharespctsharesout_1m  f__uniquebuyers_1m  \
date       tradingitemid                                                     
2004-08-06 2585893                            0.000805            1.137242   
           2585895                            0.000805            1.186371   
           2585941                            0.000805           -0.821200   
           2585957                            0.000805           -0.821200   
           2585970                            0.000805           -0.821200   
...                                                ...                 ...   
2012-11-23 586093745                          0.000805           -0.821200   
           607745150                          0.000805           -0.821200   
           610592433                          0.000805           -0.821200   
           618650356                          0.000805           -0.821200   
           636367352                          0.000805           -0.821200   

                          f__uniquesellers_1m  f__buytransactions_3m  \
date       tradingitemid                                               
2004-08-06 2585893                  -0.745811               0.667193   
           2585895                   1.368299               0.637492   
           2585941                  -0.745811              -1.488105   
           2585957                   1.287456               0.637492   
           2585970                  -0.74

### convert data to the 3D sequence data for use in LSTM

In [15]:
class To3DFeats:

    def __init__(self, labs_col=("g__12m_binary",)):
        self.idxs = []
        self.labs_col = labs_col

    def get_features3D_labels2D_lengths(self, X, y=None):
        # init
        lengths = []
        
        max_comp_num = len(X.index.get_level_values(1).unique())
        max_seq_len = len(X.index.get_level_values(0).unique())
        feat_emb_len = X.shape[1]
        
        features_3D = np.zeros((max_comp_num, max_seq_len, feat_emb_len))
        labels_2D = np.zeros((max_comp_num, max_seq_len))
        
        idx = pd.IndexSlice
        for num, compID in enumerate(X.index.get_level_values(1).unique()):
            df_feat = X.loc[idx[:, compID], idx[:]]
            features_3D[num, :len(df_feat), :] = df_feat.values
            lengths.append(len(df_feat))
            self.idxs.append(df_feat.index)
            if y is not None:
                df_labels = y.loc[idx[:, compID], idx[:]]
                labels_2D[num, :len(df_labels)] = df_labels.values.flatten()
            
        lengths = np.array(lengths)
        return features_3D, labels_2D, lengths

    def predictTodf(self, predict):
        if self.idxs:
            df = pd.DataFrame()
            start = 0
            for ind in self.idxs:
                df = pd.concat([df, pd.DataFrame(predict[start:(start + len(ind))], index=ind)])
                start += len(ind)
            df = df.sort_index()
            df.columns = self.labs_col
            return df
        else:
            raise RuntimeError("self.idxs is []. You need to run get_features3D_labels2D_lengths() first")

In [16]:
%%time
to3Dfeats_train = To3DFeats()
features_3D_train, labels_2D_train, lengths_train = to3Dfeats_train.get_features3D_labels2D_lengths(X_train, y_train)

CPU times: user 2min 22s, sys: 2.48 s, total: 2min 25s
Wall time: 2min 24s


In [17]:
features_3D_train.shape, labels_2D_train.shape, lengths_train.shape

((8335, 434, 32), (8335, 434), (8335,))

In [18]:
%%time
to3Dfeats_val = To3DFeats()
features_3D_val, labels_2D_val, lengths_val = to3Dfeats_val.get_features3D_labels2D_lengths(X_val, y_val)

CPU times: user 26.8 s, sys: 431 ms, total: 27.3 s
Wall time: 27.1 s


In [19]:
features_3D_val.shape, labels_2D_val.shape, lengths_val.shape

((4663, 109, 32), (4663, 109), (4663,))

In [20]:
%%time
to3Dfeats_test = To3DFeats()
features_3D_test, _, lengths_test = to3Dfeats_test.get_features3D_labels2D_lengths(X_test)

CPU times: user 19.8 s, sys: 337 ms, total: 20.1 s
Wall time: 20 s


In [21]:
features_3D_test.shape, lengths_test.shape

((4966, 195, 32), (4966,))

In [22]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Datasets, Dataloaders

In [24]:
class SeqDataset(Dataset):
    
    def __init__(self, features_3D, lengths, labels_2D=None):
        self.features_3D = features_3D
        if labels_2D is not None:
            self.labels_2D = labels_2D
        else:
            self.labels_2D = [0] * len(lengths)
        self.lengths = lengths
        self.data = list(zip(features_3D, lengths, self.labels_2D))

    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

In [25]:
train_dataset = SeqDataset(features_3D_train, lengths_train, labels_2D_train)
train_loader = DataLoader(train_dataset, batch_size=64)

In [26]:
val_dataset = SeqDataset(features_3D_val, lengths_val, labels_2D_val)
val_loader = DataLoader(val_dataset, batch_size=64)

In [27]:
test_dataset = SeqDataset(features_3D_test, lengths_test)
test_loader = DataLoader(test_dataset, batch_size=64)

### LSTM model

In [31]:
class Model(nn.Module):
    def __init__(self, emb_dim, hid_dim, n_layers, dropout, bidirectional):
        super().__init__()
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        self.fc1 = nn.Linear(hid_dim, hid_dim)
        self.fc2 = nn.Linear(hid_dim, 1)
        self.sigmoid = nn.Sigmoid()
        
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, inp, lens):
        inp = inp.permute(1, 0, 2)  # batch to dim=1
        packed_seq = nn.utils.rnn.pack_padded_sequence(inp, lens.cpu(), enforce_sorted=False)
        packed_output, (hidden, cell) = self.rnn(packed_seq)
        output = packed_output.data
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  
        output = output.reshape(-1, output.shape[-1])
        output = F.relu(self.fc1(output))
        output = self.fc2(output)
        output = self.sigmoid(output)
        return output

In [32]:
emb_dim = 32
hid_dim = 100
n_layers = 2
dropout = 0.5
bidirectional = False

patience = 3

max_grad_norm = 1

In [35]:
model = Model(emb_dim, hid_dim, n_layers, dropout, bidirectional).to(device)

In [36]:
optimizer = torch.optim.Adam(model.parameters())
loss_func = nn.BCELoss()

epochs = 10

### train and validation

In [37]:
min_loss = np.inf

cur_patience = 0

for epoch in range(1, epochs + 1):
    train_loss = 0.0
    train_acc = 0
    model.train()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        optimizer.zero_grad()
        X, lens, labs = batch
        X = X.float().to(device)
        labs = nn.utils.rnn.pack_padded_sequence(labs.permute(1, 0), lens.cpu(), enforce_sorted=False).data.float().to(device)
        # labs, output_lengths = nn.utils.rnn.pad_packed_sequence(labs)  
        predict = model(X, lens)
        # raise
        labs = labs.reshape(-1, 1).float().to(device)
        # print("predict.shape", predict.shape, "labs.shape", labs.shape)
        # print("lens", lens.sum())
        if (predict.shape != labs.shape):
            print("predict.shape", predict.shape, "labs.shape", labs.shape)
        loss = loss_func(predict, labs)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        train_loss += loss.item()
        train_acc += (((predict > 0.5) == labs).sum() / len(labs)).item()

    train_loss /= len(train_loader)
    train_acc /= len(train_loader)

    with torch.no_grad():
        val_loss = 0.0
        val_acc = 0
        model.eval()
        pbar = tqdm(enumerate(val_loader), total=len(val_loader), leave=False)
        pbar.set_description(f"Epoch {epoch}")
        for it, batch in pbar:
            X, lens, labs = batch
            X = X.float().to(device)
            labs = nn.utils.rnn.pack_padded_sequence(labs.permute(1, 0), lens.cpu(), enforce_sorted=False).data.float().to(device)
            predict = model(X, lens)
            labs = labs.reshape(-1, 1).float().to(device)
            loss = loss_func(predict, labs)
            val_loss += loss.item()
            val_acc += (((predict > 0.5) == labs).sum() / len(labs)).item()
        val_loss /= len(val_loader)
        val_acc /= len(val_loader)
        if val_loss < min_loss:
            min_loss = val_loss
            best_model = model.state_dict()
        else:
            cur_patience += 1
            if cur_patience == patience:
                cur_patience = 0
                break
        
        print(f'Epoch: {epoch}, Train: Loss - {train_loss}, Acc - {train_acc}, Valid: Loss - {val_loss}, Acc - {val_acc}')

Epoch: 1, Train: Loss - 0.6614502217933422, Acc - 0.5924357387400766, Valid: Loss - 0.6895886855582668, Acc - 0.5667084761678356


Epoch: 2, Train: Loss - 0.6609809016453401, Acc - 0.594835888338453, Valid: Loss - 0.6765138596704562, Acc - 0.5827489136016533


Epoch: 3, Train: Loss - 0.6595238297040226, Acc - 0.5957579262384022, Valid: Loss - 0.6764356061203839, Acc - 0.5908485320332932


Epoch: 4, Train: Loss - 0.6591690246385472, Acc - 0.5972418746420445, Valid: Loss - 0.6757805535238083, Acc - 0.5899934405333376


Epoch: 5, Train: Loss - 0.6587576383852777, Acc - 0.5982656360582541, Valid: Loss - 0.6749111511935927, Acc - 0.5870079496135451


Epoch: 6, Train: Loss - 0.6586425158813709, Acc - 0.59851432961362, Valid: Loss - 0.6750581272660869, Acc - 0.5847162714559738


Epoch: 7, Train: Loss - 0.6585256878201288, Acc - 0.5994892277335393, Valid: Loss - 0.6749314763774611, Acc - 0.5860272205039246


Epoch: 8, Train: Loss - 0.6577428379131638, Acc - 0.6017349867420342, Valid: Loss - 0.6727353424242098, Acc - 0.5907852494553344


### get test submit

In [38]:
# test
def get_test_predict(loader, model_):
    with torch.no_grad():
        predict = []
        model_.eval()
        pbar = tqdm(enumerate(loader), total=len(loader), leave=False)
        for it, batch in pbar:
            X, lens, _ = batch
            X = X.float().to(device)
            batch_pred = model_(X, lens).tolist()
            predict.extend(batch_pred)
    return predict

In [39]:
test_predict = get_test_predict(test_loader, model)

In [40]:
%%time
test_lstm_submit = to3Dfeats_test.predictTodf(test_predict)

CPU times: user 1min 12s, sys: 3.58 s, total: 1min 16s
Wall time: 1min 16s


In [41]:
test_lstm_submit.head()

g__12m_binary
date       tradingitemid               
2016-01-01 2585893             0.398152
           2585895             0.214407
           2586016             0.513938
           2586068             0.539112
           2586084             0.523694

In [42]:
test_lstm_submit.to_pickle(data_path / "test_lstm_submit.pkl")